In [5]:
pip install wordcloud nltk matplotlib pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
import nltk
from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from nltk.corpus import stopwords
from wordcloud import STOPWORDS

In [66]:
file_path = '/kaggle/input/dataset-to-identify-behavioral-disorder1/Dataset to Identify Behavioral Disorder/data_thought.csv'
df = pd.read_csv(file_path)
df

,Thought,Label
0,Everyone is probably mad at me,negative
1,You're braver than you believe,positive
2,"When I open my eyes, you're all that I want to...",positive
3,They clearly don’t want to be friends with me ...,negative
4,I've never done it before.,negative
...,...,...
156780,I can't believe how fast time flies.,negative
156781,I can't eat spicy food.,negative
156782,Itâ€™s OK to SCREW Up! Nigel Raw,positive
156783,Itâ€™s here. Itâ€™s beautiful. Itâ€™s signed. ...,positive


In [32]:
df.describe()

,Thought,Label
count,156785,156785
unique,129051,2
top,Help,positive
freq,65,85677


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156785 entries, 0 to 156784
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Thought  156785 non-null  object
 1   Label    156785 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB


In [34]:
df.shape

(156785, 2)

In [35]:
df.isnull().sum()

Thought    0
Label      0
dtype: int64

In [67]:
df['Thought'] = df['Thought'].str.lower()

In [68]:
# word_count
df['word_count'] = df['Thought'].apply(lambda x: len(str(x).split()))
# unique_word_count
df['unique_word_count'] = df['Thought'].apply(lambda x: len(set(str(x).split())))
# stop_word_count
df['stop_word_count'] = df['Thought'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
# url_count
df['url_count'] = df['Thought'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
# mean_word_length
df['mean_word_length'] = df['Thought'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# char_count
df['char_count'] = df['Thought'].apply(lambda x: len(str(x)))
# punctuation_count
df['punctuation_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
# hashtag_count
df['hashtag_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c == '#']))
# mention_count
df['mention_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c == '@']))
# Length of sentence in characters
df['length_of_Thought'] = df['Thought'].apply(lambda x: len(str(x)))
total_digits = (df['Thought'].str.findall(r'\d').apply(len)).sum()

In [69]:
df

,Thought,Label,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,length_of_Thought
0,everyone is probably mad at me,negative,6,6,3,0,4.166667,30,0,0,0,30
1,you're braver than you believe,positive,5,5,3,0,5.200000,30,1,0,0,30
2,"when i open my eyes, you're all that i want to...",positive,12,11,8,0,3.250000,50,2,0,0,50
3,they clearly don’t want to be friends with me ...,negative,10,10,5,0,4.400000,53,0,0,0,53
4,i've never done it before.,negative,5,5,1,0,4.400000,26,2,0,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...
156780,i can't believe how fast time flies.,negative,7,7,2,0,4.285714,36,2,0,0,36
156781,i can't eat spicy food.,negative,5,5,1,0,3.800000,23,2,0,0,23
156782,itâ€™s ok to screw up! nigel raw,positive,7,7,1,0,3.714286,32,1,0,0,32
156783,itâ€™s here. itâ€™s beautiful. itâ€™s signed. ...,positive,10,8,1,0,5.400000,63,4,0,0,63


In [70]:
print("BEFORE")

total_word_count = df['word_count'].sum()
print(f"Total Word Count: {total_word_count}")
total_unique_word_count = df['unique_word_count'].sum()
print(f"Total Unique Word Count: {total_unique_word_count}")
total_stop_word_count = df['stop_word_count'].sum()
print(f"Total Stop Word Count: {total_stop_word_count}")
total_url_count = df['url_count'].sum()
print(f"Total URL Count: {total_url_count}")
mean_of_mean_word_length = df['mean_word_length'].mean()
print(f"Mean of Mean Word Length: {mean_of_mean_word_length}")
total_char_count = df['char_count'].sum()
print(f"Total Character Count: {total_char_count}")
total_punctuation_count = df['punctuation_count'].sum()
print(f"Total Punctuation Count: {total_punctuation_count}")
total_hashtag_count = df['hashtag_count'].sum()
print(f"Total Hashtag # Count: {total_hashtag_count}")
total_mention_count = df['mention_count'].sum()
print(f"Total Mention @ Count: {total_mention_count}")
print(f"Total digits Count: {total_digits}")

BEFORE
Total Word Count: 1798194
Total Unique Word Count: 1701532
Total Stop Word Count: 702087
Total URL Count: 198
Mean of Mean Word Length: 5.031666509446574
Total Character Count: 10541087
Total Punctuation Count: 267464
Total Hashtag # Count: 2176
Total Mention @ Count: 552
Total digits Count: 53549


In [71]:
# df['Thought'] = df['Thought'].str.lower()
def connections(text):
    # Contractions
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"We're", "We are", text)
    text = re.sub(r"That's", "That is", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"they're", "they are", text)
    text = re.sub(r"Can't", "Cannot", text)
    text = re.sub(r"wasn't", "was not", text)
    text = re.sub(r"don\x89Ûªt", "do not", text)
    text = re.sub(r"aren't", "are not", text)
    text = re.sub(r"isn't", "is not", text)
    text = re.sub(r"What's", "What is", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"hasn't", "has not", text)
    text = re.sub(r"There's", "There is", text)
    text = re.sub(r"He's", "He is", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"you're", "you are", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"wouldn't", "would not", text)
    text = re.sub(r"i'm", "I am", text)
    text = re.sub(r"I\x89Ûªm", "I am", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"isn't", "is not", text)
    text = re.sub(r"here's", "here is", text)
    text = re.sub(r"you've", "you have", text)
    text = re.sub(r"you\x89Ûªve", "you have", text)
    text = re.sub(r"we're", "we are", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"we've", "we have", text)
    text = re.sub(r"it\x89Ûªs", "it is", text)
    text = re.sub(r"doesn\x89Ûªt", "does not", text)
    text = re.sub(r"It\x89Ûªs", "It is", text)
    text = re.sub(r"Here\x89Ûªs", "Here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"I\x89Ûªve", "I have", text)
    text = re.sub(r"y'all", "you all", text)
    text = re.sub(r"can\x89Ûªt", "cannot", text)
    text = re.sub(r"would've", "would have", text)
    text = re.sub(r"it'll", "it will", text)
    text = re.sub(r"we'll", "we will", text)
    text = re.sub(r"wouldn\x89Ûªt", "would not", text)
    text = re.sub(r"We've", "We have", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"Y'all", "You all", text)
    text = re.sub(r"Weren't", "Were not", text)
    text = re.sub(r"Didn't", "Did not", text)
    text = re.sub(r"they'll", "they will", text)
    text = re.sub(r"they'd", "they would", text)
    text = re.sub(r"DON'T", "DO NOT", text)
    text = re.sub(r"that\x89Ûªs", "That is", text)
    text = re.sub(r"they've", "they have", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"should've", "should have", text)
    text = re.sub(r"you\x89Ûªre", "You are", text)
    text = re.sub(r"You're", "You are", text)
    text = re.sub(r"you're", "You are", text)
    text = re.sub(r"youre", "you are", text)
    text = re.sub(r"ur", "you are", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"don\x89Ûªt", "Do not", text)
    text = re.sub(r"we'd", "we would", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"weren't", "were not", text)
    text = re.sub(r"they're", "They are", text)
    text = re.sub(r"can\x89Ûªt", "Can not", text)
    text = re.sub(r"you\x89Ûªll", "you will", text)
    text = re.sub(r"I\x89Ûªd", "I would", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"dont", "do not", text)
    text = re.sub(r"you're", "you are", text)
    text = re.sub(r"i've", "I have", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"doesn't", "does not", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"ain't", "am not", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"I've", "I have", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"I'll", "I will", text)
    text = re.sub(r"I'd", "I would", text)
    text = re.sub(r"let's", "Let us", text)
    text = re.sub(r"you'd", "You would", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"Ain't", "am not", text)
    text = re.sub(r"haven't", "Have not", text)
    text = re.sub(r"could've", "Could have", text)
    text = re.sub(r"youve", "you have", text)  
    text = re.sub(r"donå«t", "do not", text)
    text = re.sub(r"ain't", "is not", text)
    text = re.sub(r"could've", "could have", text)
    text = re.sub(r"might've", "might have", text)
    text = re.sub(r"must've", "must have", text)
    text = re.sub(r"should've", "should have", text)
    text = re.sub(r"would've", "would have", text)
    text = re.sub(r"they'd", "they would", text)  # or "they had" based on context
    text = re.sub(r"they'd've", "they would have", text)
    text = re.sub(r"you'd", "you would", text)  # or "you had" based on context
    text = re.sub(r"you'd've", "you would have", text)
    text = re.sub(r"I'd've", "I would have", text)
    text = re.sub(r"he'd", "he would", text)  # or "he had" based on context
    text = re.sub(r"he'd've", "he would have", text)
    text = re.sub(r"she'd", "she would", text)  # or "she had" based on context
    text = re.sub(r"she'd've", "she would have", text)
    text = re.sub(r"we'd've", "we would have", text)
    text = re.sub(r"they'd've", "they would have", text)
    return text
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
def remove_urls(text):
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'http\w+', '', text)
    text = re.sub(r'https\w+', '', text)
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)
    return text
def remove_sc(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    return text

def remove_hashtags_mentions(text):
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    return text
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in STOPWORDS])
def remove_slang(text):
    # Typos, slang and informal abbreviations
    text = re.sub(r"w/e", "whatever", text)
    text = re.sub(r"w/", "with", text)
    text = re.sub(r"USAgov", "USA government", text)
    text = re.sub(r"recentlu", "recently", text)
    text = re.sub(r"Ph0tos", "Photos", text)
    text = re.sub(r"amirite", "am I right", text)
    text = re.sub(r"exp0sed", "exposed", text)
    text = re.sub(r"<3", "love", text)
    text = re.sub(r"amageddon", "armageddon", text)
    text = re.sub(r"Trfc", "Traffic", text)
    text = re.sub(r"8/5/2015", "2015-08-05", text)
    text = re.sub(r"WindStorm", "Wind Storm", text)
    text = re.sub(r"8/6/2015", "2015-08-06", text)
    text = re.sub(r"10:38PM", "10:38 PM", text)
    text = re.sub(r"10:30pm", "10:30 PM", text)
    text = re.sub(r"16yr", "16 year", text)
    text = re.sub(r"lmao", "laughing my ass off", text)
    text = re.sub(r"TRAUMATISED", "traumatized", text)
    return text

def entity(text):
    # Character entity references
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&amp;", "&", text)
    return text

def remove_excessive_whitespace(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

 

# Apply the functions
df['Thought'] = df['Thought'].apply(connections)
df['Thought'] = df['Thought'].apply(remove_stopwords)
df['Thought'] = df['Thought'].apply(remove_slang)
df['Thought'] = df['Thought'].apply(entity)
df['Thought'] = df['Thought'].apply(remove_urls)
df['Thought'] = df['Thought'].apply(remove_hashtags_mentions)
df['Thought'] = df['Thought'].apply(remove_sc)
df['Thought'] = df['Thought'].apply(remove_punctuation)
df['Thought'] = df['Thought'].apply(remove_excessive_whitespace)
df['Thought'] = df['Thought'].str.replace(r'\d+', '', regex=True)

In [72]:
df

,Thought,Label,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,length_of_Thought
0,everyone probably mad,negative,6,6,3,0,4.166667,30,0,0,0,30
1,braver believe,positive,5,5,3,0,5.200000,30,1,0,0,30
2,open eyes want see,positive,12,11,8,0,3.250000,50,2,0,0,50
3,clearly dont want friends anymore,negative,10,10,5,0,4.400000,53,0,0,0,53
4,never done before,negative,5,5,1,0,4.400000,26,2,0,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...
156780,believe fast time flies,negative,7,7,2,0,4.285714,36,2,0,0,36
156781,eat spicy food,negative,5,5,1,0,3.800000,23,2,0,0,23
156782,itâs ok screw up nigel raw,positive,7,7,1,0,3.714286,32,1,0,0,32
156783,itâs here itâs beautiful itâs signed fits righ...,positive,10,8,1,0,5.400000,63,4,0,0,63


In [73]:
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

NaN counts per column:
Thought              0
Label                0
word_count           0
unique_word_count    0
stop_word_count      0
url_count            0
mean_word_length     0
char_count           0
punctuation_count    0
hashtag_count        0
mention_count        0
length_of_Thought    0
dtype: int64


In [74]:
nan_count = df['Thought'].isna().sum()

print(nan_count)

0


In [75]:
# word_count
df['word_count'] = df['Thought'].apply(lambda x: len(str(x).split()))
# unique_word_count
df['unique_word_count'] = df['Thought'].apply(lambda x: len(set(str(x).split())))
# stop_word_count
df['stop_word_count'] = df['Thought'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
# url_count
df['url_count'] = df['Thought'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
# mean_word_length
df['mean_word_length'] = df['Thought'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# char_count
df['char_count'] = df['Thought'].apply(lambda x: len(str(x)))
# punctuation_count
df['punctuation_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
# hashtag_count
df['hashtag_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c == '#']))
# mention_count
df['mention_count'] = df['Thought'].apply(lambda x: len([c for c in str(x) if c == '@']))
# Length of sentence in characters
df['length_of_Thought'] = df['Thought'].apply(lambda x: len(str(x)))
total_digits = (df['Thought'].str.findall(r'\d').apply(len)).sum()
df['length_of_Thought'] = df['length_of_Thought'].astype(str)
df = df[df['length_of_Thought'].apply(len) > 1]

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [77]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
# # Ensure STOPWORDS is defined
STOPWORDS = set(stopwords.words('english'))

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    """Convert treebank tags to WordNet tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text, use_stemming=False, use_lemmatization=False):
    if use_stemming:
        text = ' '.join([stemmer.stem(word) for word in text.split()])
    if use_lemmatization:
        pos_tags = pos_tag(text.split())
        text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags])
    return text

# Apply preprocessing
df['Thought'] = df['Thought'].apply(lambda x: preprocess_text(x, use_stemming=True, use_lemmatization=False))

/tmp/ipykernel_33/2608468454.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Thought'] = df['Thought'].apply(lambda x: preprocess_text(x, use_stemming=True, use_lemmatization=False))


In [76]:
print("AFTER")
total_word_count = df['word_count'].sum()
print(f"Total Word Count: {total_word_count}")
total_unique_word_count = df['unique_word_count'].sum()
print(f"Total Unique Word Count: {total_unique_word_count}")
total_stop_word_count = df['stop_word_count'].sum()
print(f"Total Stop Word Count: {total_stop_word_count}")
total_url_count = df['url_count'].sum()
print(f"Total URL Count: {total_url_count}")
mean_of_mean_word_length = df['mean_word_length'].mean()
print(f"Mean of Mean Word Length: {mean_of_mean_word_length}")
total_char_count = df['char_count'].sum()
print(f"Total Character Count: {total_char_count}")
total_punctuation_count = df['punctuation_count'].sum()
print(f"Total Punctuation Count: {total_punctuation_count}")
total_hashtag_count = df['hashtag_count'].sum()
print(f"Total Hashtag # Count: {total_hashtag_count}")
total_mention_count = df['mention_count'].sum()
print(f"Total Mention @ Count: {total_mention_count}")
print(f"Total digits Count: {total_digits}")

AFTER
Total Word Count: 1083478
Total Unique Word Count: 1061793
Total Stop Word Count: 20397
Total URL Count: 1
Mean of Mean Word Length: 6.183959909671485
Total Character Count: 7654188
Total Punctuation Count: 0
Total Hashtag # Count: 0
Total Mention @ Count: 0
Total digits Count: 0


In [78]:
df

,Thought,Label,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,length_of_Thought
0,everyon probabl mad,negative,3,3,0,0,6.333333,21,0,0,0,21
1,braver believ,positive,2,2,0,0,6.500000,14,0,0,0,14
2,open eye want see,positive,4,4,0,0,3.750000,18,0,0,0,18
3,clearli dont want friend anymor,negative,5,5,0,0,5.800000,33,0,0,0,33
4,never done befor,negative,3,3,1,0,5.000000,17,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...
156780,believ fast time fli,negative,4,4,0,0,5.000000,23,0,0,0,23
156781,eat spici food,negative,3,3,0,0,4.000000,14,0,0,0,14
156782,itâ ok screw up nigel raw,positive,6,6,1,0,3.500000,26,0,0,0,26
156783,itâ here itâ beauti itâ sign fit right in,positive,9,7,2,0,4.666667,50,0,0,0,50


In [79]:
df.describe()

,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
count,151648.000000,151648.000000,151648.000000,151648.000000,151648.000000,151648.000000,151648.0,151648.0,151648.0
mean,7.144690,7.001695,0.134502,0.000007,6.183960,50.473386,0.0,0.0,0.0
std,4.568663,4.266824,0.400050,0.002568,1.304269,33.421538,0.0,0.0,0.0
min,1.000000,1.000000,0.000000,0.000000,1.857143,10.000000,0.0,0.0,0.0
25%,4.000000,4.000000,0.000000,0.000000,5.250000,27.000000,0.0,0.0,0.0
50%,6.000000,6.000000,0.000000,0.000000,6.000000,45.000000,0.0,0.0,0.0
75%,9.000000,8.000000,0.000000,0.000000,7.000000,64.000000,0.0,0.0,0.0
max,90.000000,84.000000,8.000000,1.000000,41.000000,666.000000,0.0,0.0,0.0


In [80]:
df.isnull().sum()

Thought              0
Label                0
word_count           0
unique_word_count    0
stop_word_count      0
url_count            0
mean_word_length     0
char_count           0
punctuation_count    0
hashtag_count        0
mention_count        0
length_of_Thought    0
dtype: int64

In [81]:
df = df.dropna()

In [82]:
# Count total NaN values in the entire DataFrame
total_nan_count = df.isna().sum().sum()
print("\nTotal NaN values in the DataFrame:")
print(total_nan_count)


Total NaN values in the DataFrame:
0
